### Import data

In [39]:
include("importData.jl")
include("Hill.jl")

# import drug concentrations
param_lap_dde = CSV.read(".//figures//Dox//params_dox_DDE.csv")
concentrations = permutedims(Vector(param_lap_dde[8,2:end]));
print(concentrations)

# import G1 and G2 data
pop, g2, g1, g2_0, g1_0 = get_data("..//data//dox.csv", "..//data//dox_pop.csv");

residue(hillParams) = residHill(hillParams, concentrations, g1, g2, g1_0, g2_0)

Any[0.0 1.0 10.0 20.0 50.0 125.0 250.0 500.0]

┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, ::Colon, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = get_data(::String, ::String) at importData.jl:30
└ @ Main /home/asm/one_state_model/model/importData.jl:30


residue (generic function with 1 method)

### The new approach with the BlackBoxOptim;

In [40]:
# lower bound
low = [50.0, 0.001, 0.2, 0.01, 0.01, 0.001, 0.01, 20.0, 0.1, 0.01, 19.0, 0.01, 0.01, 0.001, 0.01, 0.001, 0.01]
# upper bound
high = [250.0, 0.01, 0.4, 0.04, 0.05, 0.02, 0.04, 35.0, 3.0, 0.04, 23.0, 2.0, 0.4, 0.01, 0.04, 0.04, 0.04]
# initial guess
guess = [100.0, 0.005, 0.3, 0.02, 0.02, 0.006, 0.02, 25.0, 1.2, 0.02, 20.0, 0.2, 0.02, 0.001, 0.02, 0.01, 0.02]

optimize_hill(concentrations, guess, low, high, g1, g2, g1_0, g2_0)

Results of Optimization Algorithm
 * Algorithm: LevenbergMarquardt
 * Minimizer: [100.84415835206788,0.0010000044597180838,0.23892380444159547,0.016323079024667675,0.010000022255511913,0.0010000038299529557,0.022255667554102212,20.356278922251423,1.2053439074036119,0.01998158229075751,19.504349083600818,0.18863696116251016,0.020090348907618183,0.0010000164920646877,0.010000003252877438,0.03999993839049507,0.010000014467840768]
 * Sum of squares at Minimum: 124057392640411888.000000
 * Iterations: 18
 * Convergence: true
 * |x - x'| < 1.0e-08: true
 * |f(x) - f(x')| / |f(x)| < 1.0e-08: false
 * |g(x)| < 1.0e-08: false
 * Function Calls: 19
 * Gradient Calls: 7
 * Multiplication Calls: 54


In [41]:
@time residue([66.906, 0.00107704, 0.266607, 0.0115474, 0.0114821, 0.01399, 0.031283, 24.3836, 2.07266, 0.0272075, 20.9898, 0.0748198, 0.151091, 0.00831905, 0.0189297, 0.0341816, 0.0275949])

  0.503342 seconds (1.86 M allocations: 179.680 MiB, 9.92% gc time)


696680.093378154

In [42]:
res = bboptimize(residue; SearchRange=collect(zip(low, high)), TraceMode=:verbose, Method = :adaptive_de_rand_1_bin_radiuslimited)

Starting optimization with optimizer DiffEvoOpt{FitPopulation{Float64},RadiusLimitedSelector,BlackBoxOptim.AdaptiveDiffEvoRandBin{3},RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
DE modify state:
0.52 secs, 2 evals, 1 steps, fitness=653468.459425541
DE modify state:
1.45 secs, 6 evals, 3 steps, fitness=653468.459425541
DE modify state:
2.41 secs, 10 evals, 5 steps, improv/step: 0.200 (last = 0.5000), fitness=653468.459425541
DE modify state:
2.97 secs, 12 evals, 6 steps, improv/step: 0.333 (last = 1.0000), fitness=653468.459425541
DE modify state:
3.48 secs, 14 evals, 7 steps, improv/step: 0.429 (last = 1.0000), fitness=524549.346713005
DE modify state:
4.01 secs, 16 evals, 8 steps, improv/step: 0.500 (last = 1.0000), fitness=524549.346713005
DE modify state:
4.93 secs, 20 evals, 10 steps, improv/step: 0.500 (last = 0.5000), fitness=524549.346713005
DE modify state:
5.53 secs, 22 evals, 11 steps, improv/step: 0.545 (last = 1.0000), fitness=524549.346713005
DE modi

┌ Warning: Optimization interrupted, recovering intermediate results...
└ @ BlackBoxOptim /home/asm/.julia/packages/BlackBoxOptim/ZdVko/src/opt_controller.jl:458


BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "InterruptException()", 281, 1.565745742130255e9, 93.74730491638184, DictChain{Symbol,Any}[DictChain{Symbol,Any}[Dict{Symbol,Any}(:RngSeed=>995326,:SearchRange=>Tuple{Float64,Float64}[(50.0, 250.0), (0.001, 0.01), (0.2, 0.4), (0.01, 0.04), (0.01, 0.05), (0.001, 0.02), (0.01, 0.04), (20.0, 35.0), (0.1, 3.0), (0.01, 0.04), (19.0, 23.0), (0.01, 2.0), (0.01, 0.4), (0.001, 0.01), (0.01, 0.04), (0.001, 0.04), (0.01, 0.04)],:TraceMode=>:verbose,:Method=>:adaptive_de_rand_1_bin_radiuslimited,:MaxSteps=>10000),Dict{Symbol,Any}()],Dict{Symbol,Any}(:FitnessScheme=>ScalarFitnessScheme{true}(),:NumDimensions=>:NotSpecified,:PopulationSize=>50,:MaxTime=>0.0,:SearchRange=>(-1.0, 1.0),:Method=>:adaptive_de_rand_1_bin_radiuslimited,:MaxNumStepsWithoutFuncEvals=>100,:RngSeed=>1234,:MaxFuncEvals=>0,:SaveTrace=>false…)], 379, ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64,Array{Float64,1}}(489327.3216407119,